<a href="https://colab.research.google.com/github/good0neighbour/ESTCampAI/blob/main/Deeplearning/03_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/ESTCampAI/Deeplearning

/content/drive/MyDrive/ESTCampAI/Deeplearning


## 목표
- 개, 고양이 이미지에 대해서 학습에 적합하게끔 이미지 데이터 전처리 수행
1. array형 변환
2. 이미지 크기 조정

In [ ]:
# 필요 라이브러리 호출
from zipfile import ZipFile # .zip 파일 다룰 때 사용하는 도구
from PIL import Image       # 이미지 파일 다룰 때 사용하는 도구
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# zip 파일 압축 해제
# 시간 걸림
zip_file_path = "./data/cats_and_dogs_filtered.zip"

with ZipFile(zip_file_path, "r") as f: # f가 할 일을 모두 완료하면 f 도구를 자동으로 닫아줌
  f.extractall(path = './data/')

In [ ]:
# 폴더 경로 지정
train_cats_dir = "./data/cats_and_dogs_filtered/train/cats"
train_dogs_dir = "./data/cats_and_dogs_filtered/train/dogs"
test_cats_dir = "./data/cats_and_dogs_filtered/test/cats"
test_dogs_dir = "./data/cats_and_dogs_filtered/test/dogs"

In [ ]:
import os

In [ ]:
# 각각 폴더에 들어있는 파일에 접근
train_cats_fn = os.listdir(train_cats_dir)
train_dogs_fn = os.listdir(train_dogs_dir)
test_cats_fn = os.listdir(test_cats_dir)
test_dogs_fn = os.listdir(test_dogs_dir)

In [ ]:
# 각 폴더에 있는 파일명을 list로 담아뒀음
# 각각 폴더에 사진 몇 장 잇는지 확인해보기
print(len(train_cats_fn), len(train_dogs_fn), len(test_cats_fn), len(test_dogs_fn))
# 내일 이미지 늘려보는 것도 해보자.

1000 1000 500 500


딥러닝 모델(CNN)은 이미지를 처리할 때 두 가지 요청 사항이 존재
1. 크기 통일 : 모든 입력 이미지가 동일한 크기여야함(ex : 모든 이미지의 크기가 28*28)
2. 숫자 변환 : 이미지 → R(Red), G(Green), B(Blue)의 픽셀 값으로 이루어진 숫자 행렬로 변환해야지만 모델 계산 시작 가능.

In [ ]:
# 이미지 한 장만 변환 -> 전체 이미지 변환.

# 1. 이미지 경로 설정
temp_path = os.path.join(train_cats_dir, train_cats_fn[150])
# 151번 이미지 파일 이름을 불러와서 폴더 위치/파일명 경로 지정 가능

# 2. 이미지 불러오기 + 리사이징(크기 통일)
temp_img = Image.open(temp_path).resize((224, 224))

# 3. 형변환
temp_img_arr = np.array(temp_img)
print(type(temp_img_arr))
print(temp_img_arr.shape)

<class 'numpy.ndarray'>
(224, 224, 3)


In [ ]:
# 모든 데이터 전처리 한번에 수행.
# 여러 이미지를 한번에 처리할 때 현재 몇 % 작업이 진행됐는지 보여주는 도구
from tqdm.auto import tqdm

In [ ]:
# 사용자 함수 정의
# 이미지 불러오기
# 이미지 resizing하기
def prepro_imgs(dir_path, file_names, img_size = (224, 224)):
  # 작업이 완료된 이미지를 담을 리스트
  imgs = []

  # 반복문 돌면서 file_name별로 전처리 작업 수행.
  for fn in tqdm(file_names):
    # 이미지 경로 설정
    path = os.path.join(dir_path, fn)
    # 이미지 불러오기 + resizing
    img = Image.open(path).resize(img_size)
    imgs.append(np.array(img)) # array로 변환 후 list에 추가

  return np.array(imgs)

In [ ]:
# 이미지 전처리 수행
X_train_cats = prepro_imgs(train_cats_dir, train_cats_fn)
# trian > dogs
X_train_dogs = prepro_imgs(train_dogs_dir, train_dogs_fn)
# test > cats
X_test_cats = prepro_imgs(test_cats_dir, test_cats_fn)
# test > dogs
X_test_dogs = prepro_imgs(test_dogs_dir, test_dogs_fn)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [23]:
X_train_cats.shape, X_train_dogs.shape, X_test_cats.shape, X_test_dogs.shape

((1000, 224, 224, 3),
 (1000, 224, 224, 3),
 (500, 224, 224, 3),
 (500, 224, 224, 3))

In [24]:
# 폴더 병합
# train 데이터에 있는 강아지 고양이 한번에 X_train으로 저장
X_train = np.concatenate((X_train_cats, X_train_dogs))
# test 데이터 병합
X_test = np.concatenate((X_test_cats, X_test_dogs))

X_train.shape, X_test.shape

((2000, 224, 224, 3), (1000, 224, 224, 3))

In [25]:
# 정답 데이터 생성
# 0 = cats, 1 = dogs
y_train = np.array([0] * 1000 + [1] * 1000)
y_test = np.array([0] * 500 + [1] * 500)

In [26]:
# npz(array 압축 파일) 만들어주기
# 여러개 넘파이 배열을 하나의 압축 파일로 만들자.
np.savez_compressed(
    "./data/np_cat_vs_dogs.npz",
    X_train = X_train,
    X_test = X_test,
    y_train = y_train,
    y_test = y_test
)